In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import cv2
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import collections


In [ ]:
def parse_voc_xml(node: ET.Element) -> Dict[str, Any]:
    voc_dict: Dict[str, Any] = {}
    children = list(node)
    if children:
        def_dic: Dict[str, Any] = collections.defaultdict(list)
        for dc in map(parse_voc_xml, children):
            for ind, v in dc.items():
                def_dic[ind].append(v)
        if node.tag == 'annotation':
            def_dic['object'] = [def_dic['object']]
        voc_dict = {
            node.tag:
                {ind: v[0] if len(v) == 1 else v
                 for ind, v in def_dic.items()}
        }
    if node.text:
        text = node.text.strip()
        if not children:
            voc_dict[node.tag] = text
    return voc_dict


def get_imgSize_and_list_of_yxyx(xml_path):
    mytree = parse_voc_xml(ET.parse(xml_path).getroot())
    xyxy=[]
    for p in mytree['annotation']['object']:
        y1=int(p['bndbox']['ymin'])
        x1=int(p['bndbox']['xmin'])
        y2=int(p['bndbox']['ymax'])
        x2=int(p['bndbox']['xmax'])
        xyxy.append(((x1,y1),(x2,y2)))
    size=(int(mytree['annotation']['size']['width']),int(mytree['annotation']['size']['height']))
    return size, xyxy



def get_xywh_from_point(size,points):
    w,h=size
    x=((points[0][0]+points[1][0])/2)/w
    y=((points[0][1]+points[1][1])/2)/h
    im_w=abs((points[0][0]-points[1][0]))/w
    im_h=abs((points[0][1]-points[1][1]))/h
    return x,y,im_w,im_h

In [ ]:
labels_dir='./data/TrainingData/labels'
anotation_path = '../../ladd-and-weights/dataset/full_train_ds/Annotations/'

os.makedirs(labels_dir,exist_ok=True)
for xml_name in os.listdir(anotation_path):
    id,_ = os.path.splitext(xml_name)
    img_size, points_yxyx=get_imgSize_and_list_of_yxyx(os.path.join(anotation_path,xml_name))
    xywhs=[get_xywh_from_point(img_size,xyxy_single) for xyxy_single in points_yxyx]
    f=open(os.path.join(labels_dir,id+'.txt'),'w')
    for box in xywhs:
        f.write(" ".join(['0',*map(str,box)])+'\n')
    f.close()

In [ ]:
dataset_images_path='./data/images'
dataset_labels_path='./data/labels'
os.makedirs(dataset_images_path,exist_ok=True)
os.makedirs(dataset_labels_path, exist_ok=True)

# train_file=open('../../ladd-and-weights/dataset/full_train_ds/ImageSets/Main/train_non_empty.txt','r')
train_file=open('../../ladd-and-weights/dataset/full_train_ds/ImageSets/Main/train.txt','r')

for id in train_file.readlines():
    id=id.strip()
    images_train_path=os.path.join(dataset_images_path,'train')
    labels_train_path=os.path.join(dataset_labels_path,'train')
    os.makedirs(images_train_path,exist_ok=True)  
    os.makedirs(labels_train_path,exist_ok=True)
    shutil.copy(os.path.join('../../ladd-and-weights/dataset/full_train_ds/JPEGImages',id+'.jpg')
              ,os.path.join(images_train_path,id+'.jpg'))
    shutil.copy(os.path.join('./data/TrainingData/labels/',id+'.txt')
              ,os.path.join(labels_train_path,id+'.txt'))    



In [ ]:
val_file=open('../../ladd-and-weights/dataset/full_train_ds/ImageSets/Main/val.txt','r')
for id in val_file.readlines():
    id=id.strip()
    images_train_path=os.path.join(dataset_images_path,'valid')
    labels_train_path=os.path.join(dataset_labels_path,'valid')
    os.makedirs(images_train_path,exist_ok=True)  
    os.makedirs(labels_train_path,exist_ok=True)
    shutil.copy(os.path.join('../../../git/ladd-and-weights/dataset/full_train_ds/JPEGImages',id+'.jpg')
              ,os.path.join(images_train_path,id+'.jpg'))
    shutil.move(os.path.join('./data/TrainingData/labels/',id+'.txt')
              ,os.path.join(labels_train_path,id+'.txt'))    
  

In [ ]:
# script for network training
# Adjust batch size, workers according to your GPU and CPU resources
# !python ./yolov5/train.py --hyp ./yolo5_settings/hyp_lacmus.yaml --batch-size 8 --worker 16 --data ./yolo5_settings/dataset_lacmus_local.yaml --weights yolov5s.pt --imgsz 1984 --epochs 50

In [1]:
# this exported without nms. Go get it with NMS see below
# cd .. & git clone https://github.com/ultralytics/yolov5
# python ./yolov5/export.py --data ./yolo5_settings/dataset_lacmus_local.yaml --weights ./exp13/weights/best.pt --img-size 1984 --dynamic --include onnx --conf-thres 0.05 --iou-thres 0.2 --batch 1
# python ./yolov5/detect.py --weights exp13/weights/best.onnx --source ../../../git/ladd-and-weights/dataset/full_train_ds/JPEGImages --imgsz 1984 --save-txt --conf-thres 0.05 --iou-thres 0.2

In [3]:
# At moment of writing only way to get nms within onnx is exporting tensorlow and then converting it to onnx (unless we want to run own code)
# https://github.com/ultralytics/yolov5/pull/5938#issuecomment-991163408
# export
# python ./yolov5/export.py --data ./yolo5_settings/dataset_lacmus_local.yaml --weights ./exp13/weights/best.pt --img-size 1984 --dynamic --include pb --conf-thres 0.05 --iou-thres 0.2 --batch 1 --nms

## validation/detection doesn't works yet, as it don't have much ideas if model exported with nms
## python ./yolov5/detect.py --weights exp13/weights/best_saved_model --source ../../../git/ladd-and-weights/dataset/full_train_ds/JPEGImages --imgsz 1984 --save-txt --conf-thres 0.05 --iou-thres 0.2

# Then you have to create onnx based on exported model
# python -m tf2onnx.convert --saved-model ./exp13/weights/best_saved_model --opset 13 --output tf2onnx_best.onnx

